In [1]:
import os
import re
from ast import literal_eval
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
import netwulf as nw
import networkx as nx
from scipy import stats as st
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx.algorithms.community as nx_comm
import colorsys
import random

In [2]:
df_texts = pd.read_csv('descriptions.csv', sep=';', converters={'properties': literal_eval})

stylized_network, config, G = nw.load("Graph.json")

In [3]:
def return_set():
    return set()
def return_list():
    return []
properties_dict = defaultdict(return_set)
for dic in df_texts['properties']:
    for el in dic.keys():
        for el1 in dic[el]:
            properties_dict[el].add(el1)

In [4]:
def generate_colors(n):
    # Set the number of color hues to evenly space the colors
    hue_values = [i/n for i in range(n)]
    saturation = 0.5
    value = 0.95

    # Convert the HSV color values to RGB format and store them in a list
    colors = []
    for hue in hue_values:
        r, g, b = colorsys.hsv_to_rgb(hue, saturation, value)
        r, g, b = int(r * 255), int(g * 255), int(b * 255)
        colors.append((r, g, b))

    # Shuffle the list to randomize the order of colors
    random.shuffle(colors)

    return colors

def graph_with_grouping(properties_dict, attribute, properties_list, Graph, return_communities=False):
    """
    Given the dictionay of properties and the property/attribute
    of interest, along with the list of properties ordered by
    node, return the graph with randomised colors.
    If return_communities, it will return a list of lists of
    communities
    :param properties_dict:
    :param attribute:
    :param Graph:
    :return:
    """
    from copy import deepcopy

    Graph = deepcopy(Graph)
    possible_values = properties_dict[attribute]
    colors_avail = generate_colors(len(possible_values))
    color_dict = {key: value for key, value in zip(possible_values, colors_avail)}
    color_list = []
    communities = defaultdict(return_list)

    for props, node in zip(properties_list, Graph.nodes):
        # Use the size to set the colour
        
        if attribute in props.keys():
            communities[props[attribute][0]].append(node)
            color = '#%02x%02x%02x' % color_dict[props[attribute][0]]
        else:
#             communities[None].append(node)
            color = '#%02x%02x%02x' % (150, 150, 150)
        color_list.append(color)
        
    # zip it up into a dictionary and set it as node attribute
    dictionaryColor = dict(zip(list(Graph.nodes), color_list))
    nx.set_node_attributes(Graph, dictionaryColor, 'group')
    
    if return_communities:
        communities = [val for val in communities.values()]
        return Graph, communities
    return Graph

def modularity(G, communities):
    m = G.number_of_edges()
    q = .0
    for c in communities:
        lc = G.subgraph(c).number_of_edges()
        kc = sum(G.degree(n) for n in c)
        q += lc / m - (kc / (2 * m)) ** 2
    return q

In [ ]:
for attribute in properties_dict.keys():
    G1, communities = graph_with_grouping(properties_dict, attribute, properties_list, G, return_communities=True)
    network, config = nw.visualize(G1, config={'zoom':0.73, 'node charge':-15, 'node gravity': 0.7, 'post to python': True})
    nx.draw(G1)
#     nw.save(f"Graph_{attribute}.json", network, config, G1)
    plt.savefig(f"Graph_{attribute}.png")
    print(f"{attribute} has modularity: {modularity(G1, communities)}")

Appearances has modularity: 0.3005900808653643
